# Explorando las localizaciones geográficas de BNB Linked Data Platform.

### Importamos las librerías de código

In [1]:
import requests
import pandas as pd
import json
import csv
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize  

### Vamos a identificar el total de registros que se encuentra enlazado a GeoNames.

In [8]:
url = 'https://bnb.data.bl.uk/sparql'
sentencia = """
PREFIX blt: <http://www.bl.uk/schemas/bibliographic/blterms#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX c4dm: <http://purl.org/NET/c4dm/event.owl#>

SELECT (count(?recurso) as ?total_obras)  WHERE {
   ?recurso dct:title ?titulo .
   ?recurso blt:publication ?publicacion .
   ?publicacion c4dm:place ?lugar .
   FILTER regex(?lugar, "geonames", "i")        
}
"""

# usamos json como resultado
cabeceras = {'Accept': 'application/sparql-results+json'}
r = requests.get(url, params = {'format': 'application/sparql-results+json', 'query': sentencia}, headers=cabeceras)

print(r.text)


{ "head": { "link": [], "vars": ["total_obras"] },
  "results": { "distinct": false, "ordered": true, "bindings": [
    { "total_obras": { "type": "typed-literal", "datatype": "http://www.w3.org/2001/XMLSchema#integer", "value": "4164098" }} ] } }


### Si eliminamos la restricción de que los registros se encuentren enlazados a GeoNames, recuperamos el total de registros

In [7]:
url = 'https://bnb.data.bl.uk/sparql'
sentencia = """
PREFIX blt: <http://www.bl.uk/schemas/bibliographic/blterms#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX c4dm: <http://purl.org/NET/c4dm/event.owl#>

SELECT (count(?recurso) as ?total_obras)  WHERE {
   ?recurso dct:title ?titulo .
   ?recurso blt:publication ?publicacion .
   ?publicacion c4dm:place ?lugar .
}
"""

# usamos json como resultado
cabeceras = {'Accept': 'application/sparql-results+json'}
r = requests.get(url, params = {'format': 'application/sparql-results+json', 'query': sentencia}, headers=cabeceras)

print(r.text)


{ "head": { "link": [], "vars": ["total_obras"] },
  "results": { "distinct": false, "ordered": true, "bindings": [
    { "total_obras": { "type": "typed-literal", "datatype": "http://www.w3.org/2001/XMLSchema#integer", "value": "8565183" }} ] } }


### Podemos comprobar que la mitad de registros (alrededor de 4 millones) de los 8 millones totales se encuentran enlazados a GeoNames.

### A continuación, vamos a recuperar el número de registros por localización geográfica en una sentencia SPARQL

In [12]:
url = 'https://bnb.data.bl.uk/sparql'
sentencia = """
PREFIX blt: <http://www.bl.uk/schemas/bibliographic/blterms#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX c4dm: <http://purl.org/NET/c4dm/event.owl#>

SELECT ?lugar (count(?recurso) as ?total_obras)  WHERE {
   ?recurso dct:title ?titulo .
   ?recurso blt:publication ?publicacion .
   ?publicacion c4dm:place ?lugar .
   FILTER regex(?lugar, "geonames", "i")        

}
GROUP BY ?lugar
"""

# usamos json como resultado
cabeceras = {'Accept': 'application/sparql-results+json'}
r = requests.get(url, params = {'format': 'application/sparql-results+json', 'query': sentencia}, headers=cabeceras)

print(r.text)


{ "head": { "link": [], "vars": ["lugar", "total_obras"] },
  "results": { "distinct": false, "ordered": true, "bindings": [
    { "lugar": { "type": "uri", "value": "http://sws.geonames.org/2635167/" }	, "total_obras": { "type": "typed-literal", "datatype": "http://www.w3.org/2001/XMLSchema#integer", "value": "362170" }},
    { "lugar": { "type": "uri", "value": "http://sws.geonames.org/6251999/" }	, "total_obras": { "type": "typed-literal", "datatype": "http://www.w3.org/2001/XMLSchema#integer", "value": "14102" }},
    { "lugar": { "type": "uri", "value": "http://sws.geonames.org/2802361/" }	, "total_obras": { "type": "typed-literal", "datatype": "http://www.w3.org/2001/XMLSchema#integer", "value": "2931" }},
    { "lugar": { "type": "uri", "value": "http://sws.geonames.org/3017382/" }	, "total_obras": { "type": "typed-literal", "datatype": "http://www.w3.org/2001/XMLSchema#integer", "value": "3750" }},
    { "lugar": { "type": "uri", "value": "http://sws.geonames.org/1269750/" }	,

### Guardamos el resultado en un fichero CSV

In [16]:
bnbdatos = json.loads(r.text)

with open('bnb_registros_total.csv', 'w', newline='') as fichero:
    csv_salida = csv.writer(fichero, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)    

    csv_salida.writerow(['id_geonames', 'total', 'lat', 'lon', 'id_wikidata'])

    for i in bnbdatos['results']['bindings']:
        id_geonames = total= lat = lon = id_wikidata=''

        id_geonames = i['lugar']['value']
        total = i['total_obras']['value']
        
        
        cadena_idgeonames =id_geonames.replace("http://sws.geonames.org/", "").replace("/", "")
        
        url_wikidata = 'https://query.wikidata.org/sparql'
        sentencia_wikidata = """
        SELECT ?idgeonames ?lat ?lon ?x ?xLabel 
        WHERE {{ 
        ?x wdt:P1566 "{0}" ; 
        p:P625 [
         psv:P625 [
           wikibase:geoLatitude ?lat ;
           wikibase:geoLongitude ?lon ;
           wikibase:geoGlobe ?globe ;
         ];
         ps:P625 ?coord
        ]
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
        }}
        """
        
        
        sentencia_wikidata = sentencia_wikidata.format(cadena_idgeonames)
        print(sentencia_wikidata)

        r_wikidata = requests.get(url_wikidata, params = {'format': 'json', 'query': sentencia_wikidata})
        puntos_geo = r_wikidata.json()
        
        
        for geo in puntos_geo['results']['bindings']:
            id_wikidata = geo['x']['value']
            lat = geo['lat']['value']
            lon = geo['lon']['value']

        csv_salida.writerow([id_geonames,total,lat,lon,id_wikidata])


        SELECT ?idgeonames ?lat ?lon ?x ?xLabel 
        WHERE { 
        ?x wdt:P1566 "2635167" ; 
        p:P625 [
         psv:P625 [
           wikibase:geoLatitude ?lat ;
           wikibase:geoLongitude ?lon ;
           wikibase:geoGlobe ?globe ;
         ];
         ps:P625 ?coord
        ]
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        }
        

        SELECT ?idgeonames ?lat ?lon ?x ?xLabel 
        WHERE { 
        ?x wdt:P1566 "6251999" ; 
        p:P625 [
         psv:P625 [
           wikibase:geoLatitude ?lat ;
           wikibase:geoLongitude ?lon ;
           wikibase:geoGlobe ?globe ;
         ];
         ps:P625 ?coord
        ]
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        }
        

        SELECT ?idgeonames ?lat ?lon ?x ?xLabel 
        WHERE { 
        ?x wdt:P1566 "2802361" ; 
        p:P625 [
         psv:P625 [
           wikibase:geoLatitude ?lat ;
 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Explorando los datos

In [17]:
df = pd.read_csv('bnb_registros_total.csv')

In [18]:
df

,id_geonames,total,lat,lon,id_wikidata
0,http://sws.geonames.org/2635167/,362170,54.600000,-2.000000,http://www.wikidata.org/entity/Q145
1,http://sws.geonames.org/6251999/,14102,56.000000,-109.000000,http://www.wikidata.org/entity/Q16
2,http://sws.geonames.org/2802361/,2931,50.641111,4.668056,http://www.wikidata.org/entity/Q31
3,http://sws.geonames.org/3017382/,3750,47.000000,2.000000,http://www.wikidata.org/entity/Q142
4,http://sws.geonames.org/1269750/,10207,22.800000,83.000000,http://www.wikidata.org/entity/Q668
5,http://sws.geonames.org/2077456/,22278,-28.000000,137.000000,http://www.wikidata.org/entity/Q408
6,http://sws.geonames.org/2634895/,73359,52.350000,-3.633333,http://www.wikidata.org/entity/Q25
7,http://sws.geonames.org/2510769/,1637,40.200000,-3.500000,http://www.wikidata.org/entity/Q29
8,http://sws.geonames.org/2328926/,89,9.000000,8.000000,http://www.wikidata.org/entity/Q1033
9,http://sws.geonames.org/798544/,722,52.000000,19.000000,http://www.wikidata.org/entity/Q36
